# Learning in Graphs from the Netzschleuder Repository

## Prerequisites

First, we need to set up our Python environment that has PyTorch, PyTorch Geometric and PathpyG installed. Depending on where you are executing this notebook, this might already be (partially) done. E.g. Google Colab has PyTorch installed by default so we only need to install the remaining dependencies. The DevContainer that is part of our GitHub Repository on the other hand already has all of the necessary dependencies installed. 

In the following, we install the packages for usage in Google Colab using Jupyter magic commands. For other environments comment in or out the commands as necessary. For more details on how to install `pathpyG` especially if you want to install it with GPU-support, we refer to our [documentation](https://www.pathpy.net/dev/getting_started/). Note that `%%capture` discards the full output of the cell to not clutter this tutorial with unnecessary installation details. If you want to print the output, you can comment `%%capture` out.

In [1]:
%%capture
# !pip install torch
!pip install torch_geometric
!pip install git+https://github.com/pathpy/pathpyG.git

## Motivation and Learning Objectives

Access to a large number of graphs with different topological characteristics and from different domains is crucial for the development and evaluation of graph learning methods. Tousands of graph data sets are available scattered throughout the web, possibly using different data formats and with missing information on their actual origin. Addressing this issue the [Netschleuder Online Repository](https://networks.skewed.de/) by Tiago Peixoto provides a single repository of graphs in a single format, including descriptions, citations, and node-/edge- or graph-level meta-data. To facilitate the development of graph learning techniques, pathpyG provides a feature that allows to directly read networks from the netzschleuder repository via an API.

In this brief unit, we will learn how we can retrieve network records and graph data from the netzschleuder repository. We will further demonstrate how we can conveniently apply a Graph Neural Network to predict node-level categories contained in the meta-data.

We first need to import a few modules.

In [1]:
import numpy as np
from matplotlib import pyplot as plt

from sklearn import metrics
from sklearn.decomposition import TruncatedSVD

import torch
from torch.nn import Linear, ReLU, Sigmoid, Parameter

import torch_geometric
from torch_geometric.nn import Sequential, GCNConv, SimpleConv, MessagePassing

import pathpyG as pp

pp.config['torch']['device'] = 'cpu'
# pp.config['torch']['device'] = 'cuda'

## Reading graphs from the netzschleuder repository

In the `pathpy.io` module, there is a function that allows to read graph data from the API. 

We can read a given networks from the netzschleuder database using its record name. Just browse the [Netschleuder Online Repository](https://networks.skewed.de/) to find the record names. In the following, we use a graph capturing co-purchase relationships between political books.



In [2]:
g = pp.io.read_netzschleuder_graph('polbooks')
print(g)

/tmp/tmpf6emfwn6
Mapping node attributes based on node indices in column `index`
Mismatch between edges in DataFrame and edges in graph
Undirected graph with 105 nodes and 441 (directed) edges

Node attributes
	node_value		<class 'numpy.ndarray'>
	node__pos		<class 'torch.Tensor'> -> torch.Size([105, 2])
	node_label		<class 'numpy.ndarray'>

Graph attributes
	transition_gap		<class 'float'>
	hashimoto_radius		<class 'float'>
	degree_assortativity		<class 'float'>
	edge_index		<class 'torch_geometric.edge_index.EdgeIndex'>
	vertex_properties		<class 'list'>
	is_bipartite		<class 'bool'>
	edge_properties		<class 'list'>
	average_degree		<class 'float'>
	is_directed		<class 'bool'>
	num_nodes		<class 'int'>
	knn_proj_1		<class 'float'>
	knn_proj_2		<class 'float'>
	mixing_time		<class 'float'>
	edge_reciprocity		<class 'float'>
	global_clustering		<class 'float'>
	num_vertices		<class 'int'>
	diameter		<class 'int'>
	degree_std_dev		<class 'float'>
	largest_component_fraction		<class 'flo

In [3]:
pp.plot(g)

In [4]:
g = pp.io.read_netzschleuder_graph('college_freshmen')
print(g)

/tmp/tmpad8o2ip0
Mapping node attributes based on node indices in column `index`
data frame contains multiple attribute values for single edge
Temporal Graph with 32 nodes, 710 unique edges and 3062 events in [913330816.0, 927846016.0]

Node attributes
	node__pos		<class 'torch.Tensor'> -> torch.Size([32, 2])

Graph attributes
	src		<class 'torch.Tensor'> -> torch.Size([3062])
	t		<class 'torch.Tensor'> -> torch.Size([3062])
	dst		<class 'torch.Tensor'> -> torch.Size([3062])



In [8]:
pp.plot(g)

If we print the resulting `Graph` instance, we find that the meta information at the node- and graph-level are automatically retrieved and added to the graph.

Let us read the famous karate club network. The record `karate club` actually contains two networks with labels `77` and `78`, which refer to two different versions of the graph data. If multiple graph data sets exist in the same record, we need to specify the name of the graph as second argument.

In [5]:
g = pp.io.read_netzschleuder_graph('karate', '78')
print(g)

/tmp/tmpwv99ufk5
Mapping node attributes based on node indices in column `index`
Mismatch between edges in DataFrame and edges in graph
Undirected graph with 34 nodes and 156 (directed) edges

Node attributes
	node__pos		<class 'torch.Tensor'> -> torch.Size([34, 2])
	node_name		<class 'torch.Tensor'> -> torch.Size([34])
	node_groups		<class 'torch.Tensor'> -> torch.Size([34])

Graph attributes
	num_nodes		<class 'int'>



In [13]:
pp.plot(g, edge_color='gray');

We see that the nodes actually contain a `node_groups` property, which maps the nodes to two groups. Those groups are often used as `ground truth` for communities in this simple illustrative graph. We will instead use it as ground truth categorical node label for a node classification experiment based on a Graph Neural Network.

In [14]:
print(g['node_name'])

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34])


In [15]:
print(g['node_groups'])

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2])


For convenience, let us shift the group labels to binary values 0 and 1: 

In [16]:
g['node_groups'] -= 1
print(g['node_groups'])

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


We can plot categorical labels by passing node colors in the pathpy plot function.

In [17]:
color_map = {0: 'red', 1: 'blue'}
colors = [ color_map[g['node_groups',v].item()] for v in g.nodes ]
pp.plot(g, node_color = colors)

## Applying Graph Neural Networks to Netzschleuder Data

We can retrieve a data object that contains the graph and its attributes:

In [ ]:
print(g.data)

Let's use a one-hot encoding of nodes as a simple feature `x`, and let's use the node groups as target label `y`. 

In [23]:
data = g.data
g.add_node_ohe('node_feature')
data['x'] = data['node_feature']
data['y'] = data['node_groups']

It is easy to define a Graph Convolutional Network that ues the one-hot-encodings of nodes and the topology to predict binary node labels:

In [ ]:
model = Sequential('node_ohe, edge_index', [
    (GCNConv(in_channels=data.num_node_features, out_channels=8), 'node_ohe, edge_index -> hidden'),
    ReLU(inplace=True),
    (GCNConv(in_channels=8, out_channels=1), 'hidden, edge_index -> output'),
    Sigmoid(),
])
model.to(pp.config['torch']['device'])

We next apply a `RandomNodeSplit` transformation to split the nodes in a training and test set.

In [25]:
transform = torch_geometric.transforms.RandomNodeSplit(split='train_rest', num_val=0.5, num_test=0)
data = transform(data)

We then train our model for 1000 epochs on the training set.

In [ ]:
epochs = 1000

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    
losses = []

model.train()
for epoch in range(epochs):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = torch.nn.functional.binary_cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    losses.append(loss.cpu().detach().numpy())

plt.plot(range(epochs), losses)
plt.grid()

We evaluate the model in the test set and calculate the adjusted mutual information for the ground truth.

In [ ]:
model.eval()
predicted_groups = model(data.x, data.edge_index).round().long()
metrics.adjusted_mutual_info_score(data.y[data.test_mask].squeeze().cpu().numpy(), predicted_groups[data.test_mask].squeeze().cpu().numpy())

We visualize node representations learned by the model. The test nodes are colored, while training nodes are greyed out.

In [ ]:
# get activations in first-layer
embedding = model[0].forward(data.x, data.edge_index)

# dimensionality reduction
svd = TruncatedSVD()
low_dim = svd.fit_transform(embedding.cpu().detach().numpy())

# plot with colors corresponding to groups in validation set
colors = {}
for v in range(g.N):
    if not data.val_mask[v]:
        colors[v] = 'grey'
    else:
        if data.y[v].item() == 0.0:
            colors[v] = 'blue'
        else:
            colors[v] = 'orange'

plt.scatter(low_dim[:,0], low_dim[:,1], c=colors.values());

This simple code gives you thousands of networks with various meta information at your fingertips, to wich you can directly apply graph learning models provided in pyG, or deep graoh learning architectures defined by yourself. 